<a href="https://colab.research.google.com/github/eathon/DeepLearningwithPython/blob/master/session8_Text_Practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Before we start...

In [30]:
import urllib

imdb_url = 'http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz'
urllib.request.urlretrieve(imdb_url, './aclImdb_v1.tar.gz') 

('./aclImdb_v1.tar.gz', <http.client.HTTPMessage at 0x7f410b743fd0>)

In [31]:
glove_url = 'http://nlp.stanford.edu/data/glove.6B.zip'
urllib.request.urlretrieve(glove_url, './glove.6B.zip')

('./glove.6B.zip', <http.client.HTTPMessage at 0x7f410b7444e0>)

In [0]:
imdb_dir = './aclImdb'
glove_dir = './glove.6B'

In [0]:
import tarfile
tar = tarfile.open('./aclImdb_v1.tar.gz', "r:gz")
tar.extractall()
tar.close()

In [0]:
import zipfile
import os
if not os.path.exists(glove_dir):
    os.mkdir(glove_dir)
zip_ref = zipfile.ZipFile('./glove.6B.zip', 'r')
zip_ref.extractall(glove_dir)
zip_ref.close()

# Text-Preprocessing

In [0]:
train_dir = os.path.join(imdb_dir, 'train')

labels = []
texts = []

for label_type in ['neg', 'pos']:
    dir_name = os.path.join(train_dir, label_type)
    for fname in os.listdir(dir_name):
        if fname[-4:] == '.txt':
            f = open(os.path.join(dir_name, fname))
            texts.append(f.read())
            f.close()
            if label_type == 'neg':
                labels.append(0)
            else:
                labels.append(1)

In [36]:
import keras
keras.__version__

'2.2.4'

In [0]:
n_data = len(texts)

In [0]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np

maxlen = 100  # We will cut reviews after 100 words
training_samples = int(n_data* 0.8) 
val_samples = n_data - training_samples
max_words = 10000  # We will only consider the top 10,000 words in the dataset

In [39]:
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 88582 unique tokens.


In [40]:
data = pad_sequences(sequences, maxlen=maxlen)

labels = np.asarray(labels)
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

# Split the data into a training set and a validation set
# But first, shuffle the data, since we started from data
# where sample are ordered (all negative first, then all positive).
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]

x_train = data[:training_samples]
y_train = labels[:training_samples]
x_val = data[training_samples: ]
y_val = labels[training_samples: ]

Shape of data tensor: (25000, 100)
Shape of label tensor: (25000,)


In [41]:
x_train.shape[0], y_train.shape[0]

(20000, 20000)

In [42]:
x_val.shape[0], y_val.shape[0]

(5000, 5000)

In [0]:
test_dir = os.path.join(imdb_dir, 'test')

labels = []
texts = []

for label_type in ['neg', 'pos']:
    dir_name = os.path.join(test_dir, label_type)
    for fname in sorted(os.listdir(dir_name)):
        if fname[-4:] == '.txt':
            f = open(os.path.join(dir_name, fname))
            texts.append(f.read())
            f.close()
            if label_type == 'neg':
                labels.append(0)
            else:
                labels.append(1)

sequences = tokenizer.texts_to_sequences(texts)
x_test = pad_sequences(sequences, maxlen=maxlen)
y_test = np.asarray(labels)

In [44]:
x_test.shape[0], y_test.shape[0]

(25000, 25000)

# Text-Vectorization

## Word Embedding

### Using pre-trained embedding (Glove)

In [45]:
embeddings_index = {}
f = open(os.path.join(glove_dir, 'glove.6B.50d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [0]:
embedding_dim = 50

embedding_matrix = np.zeros((max_words, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if i < max_words:
        if embedding_vector is not None:
            # Words not found in embedding index will be all-zeros.
            embedding_matrix[i] = embedding_vector

In [0]:
if_using_pretrained = True

# CNN 

In [48]:
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense, Convolution1D, Dropout


model_cnn = Sequential()
model_cnn.add(Embedding(max_words, embedding_dim, input_length=maxlen))
model_cnn.add(Convolution1D(2, 2))
model_cnn.add(Convolution1D(2, 3))
model_cnn.add(Flatten())
model_cnn.add(Dense(32, activation='relu'))
model_cnn.add(Dropout(0.2))
model_cnn.add(Dense(1, activation='sigmoid'))
model_cnn.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 100, 50)           500000    
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 99, 2)             202       
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 97, 2)             14        
_________________________________________________________________
flatten_2 (Flatten)          (None, 194)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 32)                6240      
_________________________________________________________________
dropout_2 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 33        
Total para

In [0]:
if if_using_pretrained:
    model_cnn.layers[0].set_weights([embedding_matrix])
    model_cnn.layers[0].trainable = False

In [50]:
model_cnn.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['acc'])
history = model_cnn.fit(x_train, y_train,
                    epochs=5,
                    batch_size=32,
                    validation_data=(x_val, y_val))

Train on 20000 samples, validate on 5000 samples
Epoch 1/5
20000/20000 [==============================] - 5s 245us/step - loss: 0.6277 - acc: 0.6478 - val_loss: 0.5883 - val_acc: 0.6882
Epoch 2/5
20000/20000 [==============================] - 4s 225us/step - loss: 0.5698 - acc: 0.7089 - val_loss: 0.5559 - val_acc: 0.7206
Epoch 3/5
20000/20000 [==============================] - 4s 223us/step - loss: 0.5550 - acc: 0.7174 - val_loss: 0.5501 - val_acc: 0.7256
Epoch 4/5
20000/20000 [==============================] - 4s 224us/step - loss: 0.5449 - acc: 0.7299 - val_loss: 0.5528 - val_acc: 0.7204
Epoch 5/5
20000/20000 [==============================] - 4s 220us/step - loss: 0.5405 - acc: 0.7320 - val_loss: 0.5488 - val_acc: 0.7258


In [51]:
model_cnn.evaluate(x_test, y_test)

25000/25000 [==============================] - 2s 71us/step


[0.556565417470932, 0.7174]

# RNN

In [0]:
from keras.layers import SimpleRNN

In [53]:
model_rnn = Sequential()
model_rnn.add(Embedding(max_words, embedding_dim, input_length=maxlen))
model_rnn.add(SimpleRNN(32, return_sequences=True))
model_rnn.add(SimpleRNN(32))  # This last layer only returns the last outputs.
model_rnn.add(Dense(1, activation='sigmoid'))
model_rnn.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 100, 50)           500000    
_________________________________________________________________
simple_rnn_3 (SimpleRNN)     (None, 100, 32)           2656      
_________________________________________________________________
simple_rnn_4 (SimpleRNN)     (None, 32)                2080      
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 33        
Total params: 504,769
Trainable params: 504,769
Non-trainable params: 0
_________________________________________________________________


In [0]:
if if_using_pretrained:
    model_rnn.layers[0].set_weights([embedding_matrix])
    model_rnn.layers[0].trainable = False

In [55]:
model_rnn.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = model_rnn.fit(x_train, y_train,
                    epochs=5,
                    batch_size=128,
                    validation_data=(x_val, y_val))

Train on 20000 samples, validate on 5000 samples
Epoch 1/5
20000/20000 [==============================] - 29s 1ms/step - loss: 0.6717 - acc: 0.5834 - val_loss: 0.6605 - val_acc: 0.6020
Epoch 2/5
20000/20000 [==============================] - 28s 1ms/step - loss: 0.6291 - acc: 0.6485 - val_loss: 0.6134 - val_acc: 0.6654
Epoch 3/5
20000/20000 [==============================] - 29s 1ms/step - loss: 0.6046 - acc: 0.6802 - val_loss: 0.6185 - val_acc: 0.6596
Epoch 4/5
20000/20000 [==============================] - 29s 1ms/step - loss: 0.6098 - acc: 0.6764 - val_loss: 0.6460 - val_acc: 0.6128
Epoch 5/5
20000/20000 [==============================] - 29s 1ms/step - loss: 0.6005 - acc: 0.6821 - val_loss: 0.6077 - val_acc: 0.6828


In [56]:
model_rnn.evaluate(x_test, y_test)

25000/25000 [==============================] - 71s 3ms/step


[0.6075507773399353, 0.68172]

# LSTM

In [57]:
from keras.layers import LSTM

model_lstm = Sequential()
model_lstm.add(Embedding(max_words, embedding_dim, input_length=maxlen))
model_lstm.add(LSTM(32))
model_lstm.add(Dense(1, activation='sigmoid'))
model_lstm.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 100, 50)           500000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 32)                10624     
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 33        
Total params: 510,657
Trainable params: 510,657
Non-trainable params: 0
_________________________________________________________________


In [0]:
if if_using_pretrained:
    model_lstm.layers[0].set_weights([embedding_matrix])
    model_lstm.layers[0].trainable = False

In [59]:
model_lstm.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['acc'])
history = model_lstm.fit(x_train, y_train,
                    epochs=5,
                    batch_size=128,
                    validation_data=(x_val, y_val))

Train on 20000 samples, validate on 5000 samples
Epoch 1/5
20000/20000 [==============================] - 42s 2ms/step - loss: 0.6324 - acc: 0.6398 - val_loss: 0.5824 - val_acc: 0.6956
Epoch 2/5
20000/20000 [==============================] - 41s 2ms/step - loss: 0.5601 - acc: 0.7149 - val_loss: 0.5944 - val_acc: 0.6890
Epoch 3/5
20000/20000 [==============================] - 42s 2ms/step - loss: 0.5322 - acc: 0.7327 - val_loss: 0.4930 - val_acc: 0.7646
Epoch 4/5
20000/20000 [==============================] - 42s 2ms/step - loss: 0.5095 - acc: 0.7507 - val_loss: 0.5800 - val_acc: 0.6946
Epoch 5/5
20000/20000 [==============================] - 41s 2ms/step - loss: 0.4891 - acc: 0.7644 - val_loss: 0.4791 - val_acc: 0.7736


In [60]:
model_lstm.evaluate(x_test, y_test)

25000/25000 [==============================] - 76s 3ms/step


[0.4908424217987061, 0.761]